# Estimate the (radiative) QED corrections for the $1s^2 2s^2 2p^2$ ground-state levels of carbon-like ions

In [1]:
using JAC


In atomic structure theory, most advanced many-electron methods are based on the *no-pair* Hamiltonian and, hence, a well-defined number of electrons. All additional processes with (virtual) photons and/or the creation and annihilation of (virtual) electron-positron pairs are at the first glance beyond these theories and need to be taken into account differently. Such a treatment is often made independently of the size of these (so-called) **radiative** or **Lamb-shift corrections**, which contribute to the binding energy roughly 1\% for (hydrogen-like) uranium ions. Of course, by far not all these radiative contributionos are immediately visible in the atomic spectra.

The *self-energy (SE)* and *vacuum polarization (VP)* are the largest contributions and need typically to  be estimated, at least, for their influence on some calculated energies, energy shift or properties and processes. Various proposals have been made in the literature to incorporate these (radiative) quantum-electrodynamic corrections by different model potentials into correlated many-electron methods, such as the MCDHF, many-body perturbation or coupled-cluster theories, for instance, by including either a *local QED Hamiltonian* **(St. Petersburg model)** or some *local QED potential* **(Sydney model)**. In the St.Petersburg model, the (radiative) QED corrections are incorporated to a good approximation into the (many-electron) representation $ \{c_r (\alpha J^P)\}$ by means of a local single-electron QED Hamiltonian that accounts additively for the SE and VP of all electrons.. 

In JAC, the incorporation of these QED estimates into the energies and wave function expansions is controlled by choosing some proper (QED) model in `AsfSettings(..)`. Apart from `qedModel = NoneQed()`, the user can select between `qedModel = QedPetersburg()` or `qedModel = QedSydney()`, and which are all singleton data structs of some abstract `AbstractQedModel`. All further details about these models can be found in the User Guide.

Let us first briefly recall the definion of `AsfSettings(..)`

In [2]:
? AsfSettings

search: AsfSettings



`struct  ManyElectron.AsfSettings`       ... a struct for defining the settings for the atomic state functions, i.e. the self-consistent-field (SCF)          and CI computations

```
+ generateScf          ::Bool               ... True, if a SCF need to be generated, and false otherwise (frozen orbitals).
+ breitScf             ::Bool               ... True, if Breit interaction is to be included into the SCF computations.
+ methodScf            ::AbstractScField    ... Specify the SCF method, for instance, Basics.ALField(), etc.
+ startScf             ::String             ... Specify how the start orbitals are obtained ["hydrogenic", "fromOrbitals"].
+ startOrbitals        ::Dict{Subshell, Orbital}  ... List of start orbitals; hydrogenic orbitals are used 
                                                      if not found in this list.
+ levelsScf            ::Array{Int64,1}     ... Levels on which the optimization need to be carried out.
+ maxIterationsScf     ::Int64              ... maximum number of SCF iterations
+ accuracyScf          ::Float64            ... convergence criterion for the SCF field.
+ shellSequenceScf     ::Array{Subshell,1}  ... Sequence of subshells to be optimized.
+ frozenSubshells      ::Array{Subshell,1}  ... Sequence of subshells to be kept frozen.

+ coulombCI            ::Bool               ... logical flag to include Coulomb interactions.
+ breitCI              ::Bool               ... logical flag to include Breit interactions.
+ qedModel             ::AbstractQedModel   ... model for estimating QED corrections {NoneQed(), QedPetersburg(), QedSydney()}.
+ methodCI             ::String             ... method for diagonalizing the matrix.
+ jjLS                 ::LSjjSettings       ... settings to control a jj-LS transformation of atomic level, if requested.

+ selectLevelsCI       ::Bool                          ... true, if specific level (number)s have been selected.
+ selectedLevelsCI     ::Array{Int64,1}                ... Level number that have been selected.
+ selectSymmetriesCI   ::Bool                          ... true, if specific level symmetries have been selected.
+ selectedSymmetriesCI ::Array{LevelSymmetry,1}        ... Level symmetries that have been selected.
```

---

`ManyElectron.AsfSettings()`  ... constructor for setting the default values.

---

`ManyElectron.AsfSettings(settings::AsfSettings;`

```
        generateScf=..,       breitScf=..,            methodScf=..,          startScf=..,               startOrbitals=..,  
        levelsScf=..,         maxIterationsScf=..,    accuracyScf=..,        shellSequenceScf=..,       frozenSubshells=..,         
        coulombCI=..,         breitCI=..,             qedModel=..,           methodCI=..,               jjLS=..,  
        selectLevelsCI=..,    selectedLevelsCI=..,    selectSymmetriesCI=.., selectedSymmetriesCI=..,   printout::Bool=false)
... constructor for re-defining a settings::AsfSettings.
```



For example, if we wish to estimate QED contributions for all levels from the $1s^2 2s^2 2p^2$ ground configuration of carbon-like Xe ions due to the QedPetersburg() model, this achieved for both, the energies and wave functions, by calling:

In [3]:
asfSettings = AsfSettings(AsfSettings(); generateScf=true, levelsScf=[1,2,3], maxIterationsScf=20, qedModel=QedPetersburg())

generateScf:          true  
breitScf:             false  
methodScf:            JAC.Basics.DFSField()  
startScf:             hydrogenic  
startOrbitals:        Dict{Subshell,Orbital}()  
levelsScf:            [1, 2, 3]  
maxIterationsScf:     20  
accuracyScf:          1.0e-6  
shellSequenceScf:     Subshell[]  
frozenSubshells:      Subshell[]  
coulombCI:            true  
breitCI:              false  
qedModel :            QedPetersburg()  
methodCI:             eigen  
jjLS :                false  
selectLevelsCI:       false  
selectedLevelsCI:     Int64[]  
selectSymmetriesCI:   false  
selectedSymmetriesCI: LevelSymmetry[]  


In [4]:
wa = Atomic.Computation(Atomic.Computation(), name="QED estimates for carbon-like Xe", 
                        grid=JAC.Radial.Grid(true), nuclearModel=Nuclear.Model(54.),
                        configs=[Configuration("1s^2 2s^2 2p^2")], asfSettings=asfSettings )
wb = perform(wa)


... in performSCF ...
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^0 2p_3/2^2 
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^1 2p_3/2^1 
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^0 
(Re-) Define a new standard subshell list.
Start SCF process with hydrogenic orbitals.
>> (Re-) Define a storage array for various B-spline matrices:
Nuclear model = Fermi nuclear model for Z = 54.0 with mass = 122.58, radius R = 4.722849091608491 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0. 
Generate hydrogenic orbital for subshell 1s_1/2 
  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      1s_1/2      -1.51935928e+03    -1.51947334e+03    +7.50765231e-05    
      2      2s_1/2      -3.83773854e+02    -3.83790162e+02    +4.24935905e-05    
 

As mentioned above, all details about the QED estimates are selected due to choice of

In [5]:
?AbstractQedModel

search: AbstractQedModel



`abstract type ManyElectron.AbstractQedModel`      ... defines an abstract and a number of singleton types for dealing with QED estimates in the many-electron         computations.

  * struct QedPetersburg          ... to estimate the QED corrections due to the model Hamiltonian by Shabaev and coworkers (2013).
  * struct QedSydney                 ... to estimate the QED corrections by means of the radiative potential by Flambaum and Ginges (2004).
  * struct NoneQed                ... No QED estimates are included into the computations.


**At present, however, only the (Sankt) Petersburg model has been implemented and tested to a reasonable degree in JAC**. Further work into this direction is welcome and readily possible.

Another simple example for computing QED estimates refer to the low-lying level and 
excitation energies of berllium-like uranium, which we first compute with the same `AsfSettings` as above and later *without QED* corrections:

In [6]:
wa = Atomic.Computation(Atomic.Computation(), name="QED estimates for beryllium-like U",  
                        grid=JAC.Radial.Grid(true), nuclearModel=Nuclear.Model(92.),
                        configs=[Configuration("1s^2 2s^2"), Configuration("1s^2 2s 2p")], asfSettings=asfSettings )
wb = perform(wa, output=true)


... in performSCF ...
>> include Configuration: 1s_1/2^2 2s_1/2^2 
>> include Configuration: 1s_1/2^2 2s_1/2^1 2p_1/2^0 2p_3/2^1 
>> include Configuration: 1s_1/2^2 2s_1/2^1 2p_1/2^1 2p_3/2^0 
(Re-) Define a new standard subshell list.
Start SCF process with hydrogenic orbitals.
>> (Re-) Define a storage array for various B-spline matrices:
Nuclear model = Fermi nuclear model for Z = 92.0 with mass = 226.32, radius R = 5.664644940594236 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0. 
Generate hydrogenic orbital for subshell 1s_1/2 
  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      1s_1/2      -4.85424576e+03    -4.86119790e+03    +1.43217757e-03    
      2      2s_1/2      -1.25607526e+03    -1.25739585e+03    +1.05136507e-03    
      3      3s_1/2

Dict{String,Any} with 2 entries:
  "grid:"      => Radial grid:  rnt = 2.0e-6,  h = 0.05,  hp = 0.0,  NoPoints =…
  "multiplet:" => name:        noName+noName+noName+noName  …

Since we performed these computations with `output=true`, we can access and store the energies by:

In [7]:
wc = wb["multiplet:"].levels

5-element Array{Level,1}:
 Level: J = 0, M = 0, parity = plus, index = 1 
energy:         -12029.752350932848  
relativeOcc:    0.0  
hasStateRep:    true  
basis:           (level.basis)  
mc:             [1.0, 0.0, 0.0, 0.0, 0.0]  
            
 Level: J = 0, M = 0, parity = minus, index = 2 
energy:         -12020.326151071236  
relativeOcc:    0.0  
hasStateRep:    true  
basis:           (level.basis)  
mc:             [0.0, 0.0, 0.0, 1.0, 0.0]  
           
 Level: J = 1, M = 1, parity = minus, index = 3 
energy:         -12018.40511229841  
relativeOcc:    0.0  
hasStateRep:    true  
basis:           (level.basis)  
mc:             [0.0, -0.0183282, 0.0, 0.0, 0.999832]  

 Level: J = 2, M = 2, parity = minus, index = 4 
energy:         -11866.562524274632  
relativeOcc:    0.0  
hasStateRep:    true  
basis:           (level.basis)  
mc:             [0.0, 0.0, 1.0, 0.0, 0.0]  
           
 Level: J = 1, M = 1, parity = minus, index = 5 
energy:         -11862.326141800972  
rel

and which we can compare with those energies without QED: 

In [8]:
newAsfSettings = AsfSettings(asfSettings, qedModel=NoneQed())
wa = Atomic.Computation(wa, asfSettings=newAsfSettings)
wb = perform(wa, output=true)


... in performSCF ...
>> include Configuration: 1s_1/2^2 2s_1/2^2 
>> include Configuration: 1s_1/2^2 2s_1/2^1 2p_1/2^0 2p_3/2^1 
>> include Configuration: 1s_1/2^2 2s_1/2^1 2p_1/2^1 2p_3/2^0 
(Re-) Define a new standard subshell list.
Start SCF process with hydrogenic orbitals.
>> (Re-) Define a storage array for various B-spline matrices:
Nuclear model = Fermi nuclear model for Z = 92.0 with mass = 226.32, radius R = 5.664644940594236 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0. 
Generate hydrogenic orbital for subshell 1s_1/2 
  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      1s_1/2      -4.85424576e+03    -4.86119790e+03    +1.43217757e-03    
      2      2s_1/2      -1.25607526e+03    -1.25739585e+03    +1.05136507e-03    
      3      3s_1/2

Dict{String,Any} with 2 entries:
  "grid:"      => Radial grid:  rnt = 2.0e-6,  h = 0.05,  hp = 0.0,  NoPoints =…
  "multiplet:" => name:        noName+noName+noName+noName  …

# **The following part of this nootebook is still under construction.**

We shall make this comparison for beryllium-like uranium more explicit to understand better at which level the QED corrections are taken into account in JAC.